# COCO Stop Sign Dataset
Steps to download COCO train, test, val dataset using fifty one library
https://cocodataset.org/#download

In [42]:
import os
if not os.path.exists("dataset"): 
    os.mkdir("dataset")

made


In [36]:
import fiftyone.zoo as foz
import fiftyone as fo

In [38]:
fo.config.dataset_zoo_dir = "/dataset"

In [ ]:
# Downloading coco dataset
# Referenced from https://medium.com/voxel51/the-coco-dataset-best-practices-for-downloading-visualization-and-evaluation-68a3d7e97fb7

train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["segmentations"],
    classes=["stop sign"],
    max_samples=400,
)

test_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="test",
    label_types=["segmentations"],
    classes=["stop sign"],
    max_samples=50,
)


val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=["stop sign"],
    max_samples=50,
)


In [4]:
from collections import defaultdict
import json
import numpy as np

# Referenced from https://machinelearningspace.com/coco-dataset-a-step-by-step-guide-to-loading-and-visualizing/
class COCOParser:
    def __init__(self, anns_file, imgs_dir):
        with open(anns_file, 'r') as f:
            coco = json.load(f)
            
        self.annIm_dict = defaultdict(list)        
        self.cat_dict = {} 
        self.annId_dict = {}
        self.im_dict = {}
        self.licenses_dict = {}
        for ann in coco['annotations']:           
            self.annIm_dict[ann['image_id']].append(ann) 
            self.annId_dict[ann['id']]=ann
        for img in coco['images']:
            self.im_dict[img['id']] = img
        for cat in coco['categories']:
            self.cat_dict[cat['id']] = cat
        for license in coco['licenses']:
            self.licenses_dict[license['id']] = license
    def get_imgIds(self):
        return list(self.im_dict.keys())
    def get_annIds(self, im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        return [ann['id'] for im_id in im_ids for ann in self.annIm_dict[im_id]]
    def load_anns(self, ann_ids):
        im_ids=ann_ids if isinstance(ann_ids, list) else [ann_ids]
        return [self.annId_dict[ann_id] for ann_id in ann_ids]        
    def load_cats(self, class_ids):
        class_ids=class_ids if isinstance(class_ids, list) else [class_ids]
        return [self.cat_dict[class_id] for class_id in class_ids]
    def get_imgLicenses(self,im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        lic_ids = [self.im_dict[im_id]["license"] for im_id in im_ids]
        return [self.licenses_dict[lic_id] for lic_id in lic_ids]

In [6]:
anns_file = "/dataset/coco-2017/validation/labels.json"
imgs_dir = "/dataset//coco-2017/validation/data"
parser = COCOParser(anns_file, imgs_dir)

In [30]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import os

stop_sign_category = 13

# Referenced from ChatGPT

def visualize_with_mask(parser, img_id, imgs_dir):
    # Load image metadata
    img_info = parser.im_dict[img_id]
    img_path = os.path.join(imgs_dir, img_info['file_name'])
    
    # Open the image
    img = Image.open(img_path).convert("RGB")
    
    # Prepare to draw segmentation masks
    draw = ImageDraw.Draw(img, "RGBA")
    
    # Load annotations for the image
    ann_ids = parser.get_annIds([img_id])
    anns = parser.load_anns(ann_ids)
    
    for ann in anns:
        if ann['category_id'] == stop_sign_category and 'segmentation' in ann:
            for seg in ann['segmentation']:
                # Convert segmentation points into pairs
                polygon = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                
                # Draw mask with transparency (e.g., red mask)
                draw.polygon(polygon, fill=(0, 255, 0, 100))  # Red with 40% opacity
                
    # Display the result
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [ ]:
img_ids = parser.get_imgIds()
visualize_with_mask(parser, img_id=img_ids[0], imgs_dir=imgs_dir)